In [1]:
import h2o
import csv
import time
import numpy as np
import matplotlib.pyplot as plt
from h2o.estimators.glrm import H2OGeneralizedLowRankEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [2]:
h2o.init()

H2O cluster uptime:,1 hours 12 minutes 23 seconds 715 milliseconds
H2O cluster version:,3.5.0.99999
H2O cluster name:,ludirehak
H2O cluster total nodes:,1
H2O cluster total memory:,4.44 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [4]:
from h2o.h2o import _locate # private function. used to find files within h2o git project directory.

# Import and parse ACS 2013 5-year DP02 demographic data
acs_orig = h2o.upload_file(path=_locate("bigdata/laptop/census/ACS_13_5YR_DP02_cleaned.zip"), col_types = (["enum"] + ["numeric"]*149))
acs_orig.describe()

acs_zcta_col = acs_orig["ZCTA5"].asfactor()
acs_full = acs_orig.drop("ZCTA5")


Parse Progress: [##################################################] 100%
Uploaded py3e35ed84-d1df-4f1b-8b38-3eb5f9a0348d into cluster with 32,989 rows and 150 cols
Rows: 32,989 Cols: 150

Chunk compression summary:


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C2,2-Byte Integers,104,69.333336,6.6 MB,60.1219
C2S,2-Byte Fractions,23,15.333334,1.4 MB,13.29941
C4,4-Byte Integers,23,15.333334,2.9 MB,26.578688



Frame distribution summary:


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.17:54321,10.9 MB,32989.0,1.0,150.0
mean,10.9 MB,32989.0,1.0,150.0
min,10.9 MB,32989.0,1.0,150.0
max,10.9 MB,32989.0,1.0,150.0
stddev,0 B,0.0,0.0,0.0
total,10.9 MB,32989.0,1.0,150.0



Column-by-Column Summary:



,ZCTA5,total_households,family_households,families_w_children_under_18,married_families,married_w_children_under_18,male_no_wife,male_no_wife_w_children_under_18,female_no_husband,female_no_husband_w_children_under_18,nonfamily_households,living_alone,living_alone_65_and_over,households_w_people_under_18,households_w_people_over_65,average_household_size,average_family_size,pop_in_households,pop_householder,pop_spouse,pop_child,pop_other_relatives,pop_nonrelatives,pop_unmarried_partner,males_15_and_over,males_15_and_over_unmarried,males_15_and_over_married,males_15_and_over_separated,males_15_and_over_widowed,males_15_and_over_divorced,females_15_and_over,females_15_and_over_unmarried,females_15_and_over_married,females_15_and_over_separated,females_15_and_over_widowed,females_15_and_over_divorced,women_birth_past_year,unmarried_women_birth_past_year,unmarried_women_birth_past_year_per_1000,women_birth_past_year_per_1000,women_15_to_19_birth_past_year_per_1000,women_20_to_34_birth_past_year_per_1000,women_35_to_50_birth_past_year_per_1000,grdparents_live_w_grdchildren,grdparents_live_w_resp_grdchildren,resp_grdchildren_less_1yr,resp_grdchildren_1_or_2yrs,resp_grdchildren_3_or_4yrs,resp_grdchildren_5yrs_or_more,grdparents_resp_grdchildren,female_grdparents_resp_grdchildren,married_grdparents_resp_grdchildren,pop_3_and_over_in_school,nursery_and_preschool,kindergarten,elementary_school,high_school,college_or_graduate_school,pop_25_years_and_over,less_than_9th_grade,high_school_no_diploma,high_school_graduate,college_no_degree,associate_degree,bachelors_degree,graduate_or_prof_degree,pct_high_school_grad_or_higher,pct_bachelors_degree_or_higher,civilian_18_and_over,civilian_veterans,pop_civilian_noninst,pop_civilian_w_disability,age_under_18,age_under_18_w_disability,age_18_to_64,age_18_to_64_w_disability,age_65_and_over,age_65_and_over_w_disability,pop_1_and_over,res_same_house,res_diff_house_in_us,res_same_county,res_diff_county,res_diff_county_same_state,res_diff_county_and_state,res_abroad,total_pop_place_of_birth,pop_native,native_born_in_us,born_state_of_residence,born_different_state,born_islands_american_abroad,total_pop_foreign_born,pop_foreign_born,foreign_born_naturalized_us_citizen,foreign_born_not_us_citizen,pop_born_outside_us,native,native_entered_2010_or_later,native_entered_before_2010,foreign_born,foreign_born_entered_2010_or_later,foreign_born_entered_before_2010,foreign_born_excl_born_at_sea,foreign_born_europe,foreign_born_asia,foreign_born_africa,foreign_born_oceania,foreign_born_latin_america,foreign_born_northern_america,pop_5_and_over,english_language_only,non_english_language,non_english_poor_english,spanish_language,spanish_poor_english,other_indo_euro_language,other_indo_euro_poor_english,asian_language,asian_poor_english,other_language,english_less_than_v_well,total_pop_ancestory,pop_american,pop_arab,pop_czech,pop_danish,pop_dutch,pop_english,pop_french,pop_french_canadian,pop_german,pop_greek,pop_hungarian,pop_irish,pop_italian,pop_lithuanian,pop_norwegian,pop_polish,pop_portuguese,pop_russian,pop_scotch_irish,pop_scottish,pop_slovak,pop_subsaharan_africa,pop_swedish,pop_swiss,pop_ukrainian,pop_welsh,pop_west_indian
type,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int
mins,1001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [27]:
# Import and parse WHD 2014-2015 labor violations data
whd_zcta = h2o.upload_file(path=_locate("bigdata/laptop/census/whd_zcta_cleaned.zip"), col_types = (["enum"]*7 + ["numeric"]*97))
whd_zcta["zcta5_cd"] = whd_zcta["zcta5_cd"].asfactor()
whd_zcta.describe()


Parse Progress: [##################################################] 100%
Uploaded py5891cb5a-3822-4778-b7fc-e2b800061a24 into cluster with 208,806 rows and 104 cols
Rows: 208,806 Cols: 104

Chunk compression summary:


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C0L,Constant Integers,40,19.23077,3.1 KB,0.0
CX0,Sparse Bits,7,3.3653848,674 B,0.0
CXI,Sparse Integers,69,33.173077,213.7 KB,1.4656583
C1,1-Byte Integers,4,1.923077,408.1 KB,2.7992976
C1N,1-Byte Integers (w/o NAs),3,1.4423077,308.8 KB,2.1181762
C2,2-Byte Integers,13,6.25,2.6 MB,17.927673
C4,4-Byte Integers,14,6.7307696,5.6 MB,39.470284
C4S,4-Byte Fractions,12,5.769231,4.8 MB,33.576458
CXD,Sparse Reals,46,22.115383,381.4 KB,2.6165023



Frame distribution summary:


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.17:54321,14.2 MB,208806.0,2.0,208.0
mean,14.2 MB,208806.0,2.0,208.0
min,14.2 MB,208806.0,2.0,208.0
max,14.2 MB,208806.0,2.0,208.0
stddev,0 B,0.0,0.0,0.0
total,14.2 MB,208806.0,2.0,208.0



Column-by-Column Summary:



,case_id,trade_nm,cty_nm,st_cd,zcta5_cd,naic_cd,flsa_repeat_violator,case_violtn_cnt,cmp_assd_cnt,ee_violtd_cnt,bw_atp_amt,ee_atp_cnt,flsa_violtn_cnt,flsa_bw_atp_amt,flsa_ee_atp_cnt,flsa_mw_bw_atp_amt,flsa_ot_bw_atp_amt,flsa_15a3_bw_atp_amt,flsa_cmp_assd_amt,sca_violtn_cnt,sca_bw_atp_amt,sca_ee_atp_cnt,mspa_violtn_cnt,mspa_bw_atp_amt,mspa_ee_atp_cnt,mspa_cmp_assd_amt,h1b_violtn_cnt,h1b_bw_atp_amt,h1b_ee_atp_cnt,h1b_cmp_assd_amt,fmla_violtn_cnt,fmla_bw_atp_amt,fmla_ee_atp_cnt,fmla_cmp_assd_amt,flsa_cl_violtn_cnt,flsa_cl_minor_cnt,flsa_cl_cmp_assd_amt,dbra_cl_violtn_cnt,dbra_bw_atp_amt,dbra_ee_atp_cnt,h2a_violtn_cnt,h2a_bw_atp_amt,h2a_ee_atp_cnt,h2a_cmp_assd_amt,flsa_smw14_violtn_cnt,flsa_smw14_bw_amt,flsa_smw14_ee_atp_cnt,cwhssa_violtn_cnt,cwhssa_bw_amt,cwhssa_ee_cnt,osha_violtn_cnt,osha_bw_atp_amt,osha_ee_atp_cnt,osha_cmp_assd_amt,eppa_violtn_cnt,eppa_bw_atp_amt,eppa_ee_cnt,eppa_cmp_assd_amt,h1a_violtn_cnt,h1a_bw_atp_amt,h1a_ee_atp_cnt,h1a_cmp_assd_amt,crew_violtn_cnt,crew_bw_atp_amt,crew_ee_atp_cnt,crew_cmp_assd_amt,ccpa_violtn_cnt,ccpa_bw_atp_amt,ccpa_ee_atp_cnt,flsa_smwpw_violtn_cnt,flsa_smwpw_bw_atp_amt,flsa_smwpw_ee_atp_cnt,flsa_hmwkr_violtn_cnt,flsa_hmwkr_bw_atp_amt,flsa_hmwkr_ee_atp_cnt,flsa_hmwkr_cmp_assd_amt,ca_violtn_cnt,ca_bw_atp_amt,ca_ee_atp_cnt,pca_violtn_cnt,pca_bw_atp_amt,pca_ee_atp_cnt,flsa_smwap_violtn_cnt,flsa_smwap_bw_atp_amt,flsa_smwap_ee_atp_cnt,flsa_smwft_violtn_cnt,flsa_smwft_bw_atp_amt,flsa_smwft_ee_atp_cnt,flsa_smwl_violtn_cnt,flsa_smwl_bw_atp_amt,flsa_smwl_ee_atp_cnt,flsa_smwmg_violtn_cnt,flsa_smwmg_bw_atp_amt,flsa_smwmg_ee_atp_cnt,flsa_smwsl_violtn_cnt,flsa_smwsl_bw_atp_amt,flsa_smwsl_ee_atp_cnt,eev_violtn_cnt,h2b_violtn_cnt,h2b_bw_atp_amt,h2b_ee_atp_cnt,sraw_violtn_cnt,sraw_bw_atp_amt,sraw_ee_atp_cnt
type,int,enum,enum,enum,enum,int,enum,int,real,int,real,int,int,real,int,real,real,real,real,int,real,int,int,real,int,real,int,real,int,real,int,real,int,int,int,int,real,int,real,int,int,real,int,real,int,real,int,int,real,int,int,int,int,real,int,real,int,int,int,real,int,int,int,int,int,int,int,real,int,int,real,int,int,real,int,int,int,real,int,int,real,int,int,int,int,int,real,int,int,int,int,int,int,int,int,real,int,int,int,real,int,int,int,int
mins,1918.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
maxs,1081028538.0,178147.0,11043.0,55.0,19358.0,928120.0,3.0,530002.0,3129900.0,76664.0,30857204.7,76664.0,76664.0,30857204.7,76664.0,1426727.4,30857204.7,103847.31,708950.0,13667.0,18598525.9,7170.0,905.0,123330.54,903.0,97225.0,1742.0,4222146.3,1046.0,1740000.0,530002.0,118250.0,105.0,100.0,766.0,764.0,287980.0,752.0,5745385.3,747.0,2661.0,2338699.6,1365.0,3129900.0,544.0,299592.9,544.0,6171.0,3213935.4,2153.0,32.0,0.0,0.0,37800.0,166.0,20750.78,2.0,274000.0,119.0,1041759.0,119.0,40000.0,2.0,0.0,0.0,165000.0,27.0,25000.0,27.0,220.0,188895.49,191.0,43.0,15945.66,33.0,500.0,45.0,25973.23,45.0,224.0,360000.0,224.0,0.0,0.0,0.0,7.0,1254.91,7.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,382.05,2.0,1.0,194.0,93507.0,159.0,0.0,0.0,0.0
mean,1554203.5,NaN,NaN,NaN,NaN,386060.0,NaN,26.7,539.6,13.8,10330.6,12.5,13.3,7610.0,10.9,1092.4,6512.0,2.5,209.4,1.2,1262.5,0.7,0.3,21.3,0.1,45.4,0.1,347.5,0.0,35.9,10.1,48.9,0.0,0.0,0.2,0.1,154.2,0.5,781.1,0.3,0.3,63.1,0.1,76.6,0.3,44.2,0.3,0.3,135.7,0.3,0.0,0.0,0.0,3.1,0.0,0.7,0.0,6.3,0.0,5.0,0.0,0.2,0.0,0.0,0.0,1.4,0.0,0.5,0.0,0.0,2.2,0.0,0.0,0.2,0.0,0.0,0.0,0.3,0.0,0.0,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.1,0.0,0.0,0.0,0.0
sigma,2365752.3,NaN,NaN,NaN,NaN,269790.4,NaN,1404.6,10479.0,198.0,105051.0,197.4,199.2,84369.4,196.0,12170.2,83106.5,292.7,4025.9,37.1,51171.7,21.2,5.9,804.5,4.9,69

In [28]:
# Run GLRM to reduce ZCTA demographics to 10 archetypes
acs_model = H2OGeneralizedLowRankEstimator(x = acs_full, 
                                           k = 10,
                                           transform = "STANDARDIZE",
                                           loss = "Quadratic",
                                           regularization_x = "Quadratic",
                                           regularization_y = "L1",
                                           gamma_x = 0.25,
                                           gamma_y = 0.5,
                                           max_iterations = 100)
print acs_model


glrm Model Build Progress: [##################################################] 100%
Model Details
H2ODimReductionModel :  Beta - Generalized Low Rank Model
Model Key:  GLRM_model_python_1445542366405_12

Model Summary:


,number_of_iterations,final_step_size,final_objective_value
,100.0,0.0,1176209.8




ModelMetricsGLRM: glrm
** Reported on train data. **

MSE: NaN
Sum of Squared Error (Numeric): 1169959.17186
Misclassification Error (Categorical): 0.0

Scoring History:


,timestamp,duration,iteration,step_size,objective
,2015-10-22 14:36:20,23.419 sec,0.0,0.7,2385272.8
,2015-10-22 14:36:22,25.864 sec,1.0,0.4,2385272.8
,2015-10-22 14:36:24,28.209 sec,2.0,0.2,2385272.8
,2015-10-22 14:36:27,30.716 sec,3.0,0.1,2385272.8
,2015-10-22 14:36:29,33.124 sec,4.0,0.0,2385272.8
---,---,---,---,---,---
,2015-10-22 14:40:18,4 min 21.377 sec,95.0,0.0,1179785.1
,2015-10-22 14:40:21,4 min 24.380 sec,96.0,0.0,1178595.4
,2015-10-22 14:40:23,4 min 26.800 sec,97.0,0.0,1178577.6
,2015-10-22 14:40:25,4 min 29.204 sec,98.0,0.0,1178577.6


In [29]:
# Plot objective function value each iteration
acs_model_score = acs_model.score_history()
plt.xlabel("Iteration")
plt.ylabel("Objective")
plt.title("Objective Function Value per Iteration")
plt.plot(acs_model_score["iteration"], acs_model_score["objective"])
plt.show()

In [30]:
# Embedding of ZCTAs into archetypes (X)
zcta_arch_x = h2o.get_frame(acs_model._model_json["output"]["representation_name"])
zcta_arch_x.head()

H2OFrame with 32989 rows and 10 columns: 


Arch1,Arch2,Arch3,Arch4,Arch5,Arch6,Arch7,Arch8,Arch9,Arch10
0.1,0.0,-0.1,0.1,0.0,-0.0,0.3,0.1,0.2,0.0
0.1,0.7,-0.1,-0.1,0.0,0.0,0.2,0.0,0.1,0.1
0.3,0.3,-0.1,-0.1,0.0,-0.0,0.0,-0.0,0.3,-0.0
0.4,0.0,-0.0,-0.0,-0.0,-0.0,0.1,-0.0,0.4,0.0
0.2,0.0,-0.0,-0.0,0.0,-0.0,0.3,0.0,0.3,0.0
0.9,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0
0.9,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0
0.7,0.0,-0.0,-0.0,-0.0,-0.0,0.1,-0.0,0.2,0.0
0.9,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0
0.9,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.1,-0.0


In [31]:
# BUG: The or command isn't working, idx only contains indicator of "10065"
# Plot a few ZCTAs on the first two archetypes
idx = ((acs_zcta_col == "10065") |   # Manhattan, NY (Upper East Side)
       (acs_zcta_col == "11219") |   # Manhattan, NY (East Harlem)
       (acs_zcta_col == "66753") |   # McCune, KS
       (acs_zcta_col == "84104") |   # Salt Lake City, UT
       (acs_zcta_col == "94086") |   # Sunnyvale, CA
       (acs_zcta_col == "95014"))    # Cupertino, CA

city_arch = np.array(h2o.as_list(zcta_arch_x[idx,[0,1]]))
plt.xlabel("First Archetype")
plt.ylabel("Second Archetype")
plt.title("Archetype Representation of Zip Code Tabulation Areas")
plt.plot(city_arch[:,0], city_arch[:,1], "o")

# Label city names corresponding to ZCTAs
city_names = ["Upper East Side", "East Harlem", "McCune", "Salt Lake City", "Sunnyvale", "Cupertino"]
for i, txt in enumerate(city_names):
   plt.annotate(txt, (city_arch[i,0], city_arch[i,1]))
plt.show()

In [32]:
# Archetypes to full feature mapping (Y)
arch_feat_y = acs_model._model_json["output"]["archetypes"]
print arch_feat_y


Archetypes:


,total_households,family_households,families_w_children_under_18,married_families,married_w_children_under_18,male_no_wife,male_no_wife_w_children_under_18,female_no_husband,female_no_husband_w_children_under_18,nonfamily_households,living_alone,living_alone_65_and_over,households_w_people_under_18,households_w_people_over_65,average_household_size,average_family_size,pop_in_households,pop_householder,pop_spouse,pop_child,pop_other_relatives,pop_nonrelatives,pop_unmarried_partner,males_15_and_over,males_15_and_over_unmarried,males_15_and_over_married,males_15_and_over_separated,males_15_and_over_widowed,males_15_and_over_divorced,females_15_and_over,females_15_and_over_unmarried,females_15_and_over_married,females_15_and_over_separated,females_15_and_over_widowed,females_15_and_over_divorced,women_birth_past_year,unmarried_women_birth_past_year,unmarried_women_birth_past_year_per_1000,women_birth_past_year_per_1000,women_15_to_19_birth_past_year_per_1000,women_20_to_34_birth_past_year_per_1000,women_35_to_50_birth_past_year_per_1000,grdparents_live_w_grdchildren,grdparents_live_w_resp_grdchildren,resp_grdchildren_less_1yr,resp_grdchildren_1_or_2yrs,resp_grdchildren_3_or_4yrs,resp_grdchildren_5yrs_or_more,grdparents_resp_grdchildren,female_grdparents_resp_grdchildren,married_grdparents_resp_grdchildren,pop_3_and_over_in_school,nursery_and_preschool,kindergarten,elementary_school,high_school,college_or_graduate_school,pop_25_years_and_over,less_than_9th_grade,high_school_no_diploma,high_school_graduate,college_no_degree,associate_degree,bachelors_degree,graduate_or_prof_degree,pct_high_school_grad_or_higher,pct_bachelors_degree_or_higher,civilian_18_and_over,civilian_veterans,pop_civilian_noninst,pop_civilian_w_disability,age_under_18,age_under_18_w_disability,age_18_to_64,age_18_to_64_w_disability,age_65_and_over,age_65_and_over_w_disability,pop_1_and_over,res_same_house,res_diff_house_in_us,res_same_county,res_diff_county,res_diff_county_same_state,res_diff_county_and_state,res_abroad,total_pop_place_of_birth,pop_native,native_born_in_us,born_state_of_residence,born_different_state,born_islands_american_abroad,total_pop_foreign_born,pop_foreign_born,foreign_born_naturalized_us_citizen,foreign_born_not_us_citizen,pop_born_outside_us,native,native_entered_2010_or_later,native_entered_before_2010,foreign_born,foreign_born_entered_2010_or_later,foreign_born_entered_before_2010,foreign_born_excl_born_at_sea,foreign_born_europe,foreign_born_asia,foreign_born_africa,foreign_born_oceania,foreign_born_latin_america,foreign_born_northern_america,pop_5_and_over,english_language_only,non_english_language,non_english_poor_english,spanish_language,spanish_poor_english,other_indo_euro_language,other_indo_euro_poor_english,asian_language,asian_poor_english,other_language,english_less_than_v_well,total_pop_ancestory,pop_american,pop_arab,pop_czech,pop_danish,pop_dutch,pop_english,pop_french,pop_french_canadian,pop_german,pop_greek,pop_hungarian,pop_irish,pop_italian,pop_lithuanian,pop_norwegian,pop_polish,pop_portuguese,pop_russian,pop_scotch_irish,pop_scottish,pop_slovak,pop_subsaharan_africa,pop_swedish,pop_swiss,pop_ukrainian,pop_welsh,pop_west_indian
Arch1,-0.4,-0.5,-0.3,-0.5,-0.4,-0.3,-0.4,-0.4,-0.4,-0.5,-0.5,-0.5,-0.4,-0.5,-0.1,-0.1,-0.4,-0.4,-0.5,-0.4,-0.3,-0.3,-0.4,-0.5,-0.4,-0.5,-0.3,-0.5,-0.6,-0.5,-0.3,-0.5,-0.3,-0.5,-0.5,-0.3,-0.3,-0.3,-0.3,-0.3,-0.2,-0.2,-0.3,-0.4,-0.3,-0.3,-0.4,-0.3,-0.4,-0.4,-0.4,-0.4,-0.4,-0.3,-0.4,-0.4,-0.3,-0.5,-0.0,-0.3,-0.5,-0.6,-0.5,-0.4,-0.3,0.0,-0.4,-0.5,-0.7,-0.4,-0.5,-0.4,-0.5,-0.3,-0.5,-0.5,-0.5,-0.4,-0.4,-0.5,-0.4,-0.5,-0.5,-0.4,-0.1,-0.4,-0.6,-0.6,-0.5,-0.6,-0.0,0.1,0.1,0.1,0.0,0.1,-0.0,0.0,-0.0,0.1,0.1,0.1,0.1,0.1,0.3,0.0,-0.1,0.0,-0.2,-0.4,-0.7,0.1,0.0,-0.0,0.0,0.3,0.5,0.2,0.2,0.2,0.3,-0.4,-0.6,0.2,-0.3,-0.3,-0.4,-0.7,-0.5,-0.3,-0.6,-0.2,0.4,-0.6,-0.3,-0.1,-0.3,-0.2,-0.1,0.1,-0.6,-0.7,-0.2,-0.1,-0.4,-0.5,-0.0,-0.5,0.1
Arch2,2.7,1.7,2.0,1.5,1.8,1.9,1.5,1.9,1.9,4.0,3.4,1.3,1.8,1.1,-0.4,0.0,2.3,2.7,1.

In [33]:
# Split WHD data into test/train with 20/80 ratio
split = whd_zcta["flsa_repeat_violator"].runif()
train = whd_zcta[split <= 0.8]
test = whd_zcta[split > 0.8]

# Build a DL model to predict repeat violators and score
s = time.time()
dl_orig = H2ODeepLearningEstimator(epochs = 0.1, hidden = [50,50,50], distribution = "multinomial")
idx_x = range(4,150)
idx_x.remove(train.names.index("flsa_repeat_violator"))

dl_orig.train(x               =idx_x,
              y               ="flsa_repeat_violator",
              training_frame  =train,
              validation_frame=test)
orig_elapsed = time.time() - s


deeplearning Model Build Progress: [##################################################] 100%


In [34]:
# Replace zcta5_cd column in WHD data with GLRM archetypes
zcta_arch_x["zcta5_cd"] = acs_zcta_col
whd_arch = whd_zcta.merge(zcta_arch_x, allLeft = True, allRite = False)
whd_arch = whd_arch.drop("zcta5_cd")
whd_arch.describe()

Rows: 208,806 Cols: 113

Chunk compression summary:


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C0L,Constant Integers,40,17.699116,3.1 KB,0.0
CX0,Sparse Bits,7,3.097345,674 B,0.0
CXI,Sparse Integers,69,30.530973,213.7 KB,0.6039514
C1,1-Byte Integers,2,0.9,204.0 KB,0.6
C1N,1-Byte Integers (w/o NAs),5,2.2123895,512.8 KB,1.4495845
C2,2-Byte Integers,11,4.8672566,2.2 MB,6.2343006
C4,4-Byte Integers,14,6.19469,5.6 MB,16.264456
CXD,Sparse Reals,46,20.353981,381.4 KB,1.0781779
C8D,64-bit Reals,32,14.159292,25.5 MB,73.78209



Frame distribution summary:


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.17:54321,34.5 MB,208806.0,2.0,226.0
mean,34.5 MB,208806.0,2.0,226.0
min,34.5 MB,208806.0,2.0,226.0
max,34.5 MB,208806.0,2.0,226.0
stddev,0 B,0.0,0.0,0.0
total,34.5 MB,208806.0,2.0,226.0



Column-by-Column Summary:



,trade_nm,cty_nm,st_cd,case_id,naic_cd,flsa_repeat_violator,case_violtn_cnt,cmp_assd_cnt,ee_violtd_cnt,bw_atp_amt,ee_atp_cnt,flsa_violtn_cnt,flsa_bw_atp_amt,flsa_ee_atp_cnt,flsa_mw_bw_atp_amt,flsa_ot_bw_atp_amt,flsa_15a3_bw_atp_amt,flsa_cmp_assd_amt,sca_violtn_cnt,sca_bw_atp_amt,sca_ee_atp_cnt,mspa_violtn_cnt,mspa_bw_atp_amt,mspa_ee_atp_cnt,mspa_cmp_assd_amt,h1b_violtn_cnt,h1b_bw_atp_amt,h1b_ee_atp_cnt,h1b_cmp_assd_amt,fmla_violtn_cnt,fmla_bw_atp_amt,fmla_ee_atp_cnt,fmla_cmp_assd_amt,flsa_cl_violtn_cnt,flsa_cl_minor_cnt,flsa_cl_cmp_assd_amt,dbra_cl_violtn_cnt,dbra_bw_atp_amt,dbra_ee_atp_cnt,h2a_violtn_cnt,h2a_bw_atp_amt,h2a_ee_atp_cnt,h2a_cmp_assd_amt,flsa_smw14_violtn_cnt,flsa_smw14_bw_amt,flsa_smw14_ee_atp_cnt,cwhssa_violtn_cnt,cwhssa_bw_amt,cwhssa_ee_cnt,osha_violtn_cnt,osha_bw_atp_amt,osha_ee_atp_cnt,osha_cmp_assd_amt,eppa_violtn_cnt,eppa_bw_atp_amt,eppa_ee_cnt,eppa_cmp_assd_amt,h1a_violtn_cnt,h1a_bw_atp_amt,h1a_ee_atp_cnt,h1a_cmp_assd_amt,crew_violtn_cnt,crew_bw_atp_amt,crew_ee_atp_cnt,crew_cmp_assd_amt,ccpa_violtn_cnt,ccpa_bw_atp_amt,ccpa_ee_atp_cnt,flsa_smwpw_violtn_cnt,flsa_smwpw_bw_atp_amt,flsa_smwpw_ee_atp_cnt,flsa_hmwkr_violtn_cnt,flsa_hmwkr_bw_atp_amt,flsa_hmwkr_ee_atp_cnt,flsa_hmwkr_cmp_assd_amt,ca_violtn_cnt,ca_bw_atp_amt,ca_ee_atp_cnt,pca_violtn_cnt,pca_bw_atp_amt,pca_ee_atp_cnt,flsa_smwap_violtn_cnt,flsa_smwap_bw_atp_amt,flsa_smwap_ee_atp_cnt,flsa_smwft_violtn_cnt,flsa_smwft_bw_atp_amt,flsa_smwft_ee_atp_cnt,flsa_smwl_violtn_cnt,flsa_smwl_bw_atp_amt,flsa_smwl_ee_atp_cnt,flsa_smwmg_violtn_cnt,flsa_smwmg_bw_atp_amt,flsa_smwmg_ee_atp_cnt,flsa_smwsl_violtn_cnt,flsa_smwsl_bw_atp_amt,flsa_smwsl_ee_atp_cnt,eev_violtn_cnt,h2b_violtn_cnt,h2b_bw_atp_amt,h2b_ee_atp_cnt,sraw_violtn_cnt,sraw_bw_atp_amt,sraw_ee_atp_cnt,Arch2,Arch3,Arch4,Arch5,Arch6,Arch7,Arch8,Arch9,Arch10,Arch1
type,enum,enum,enum,int,int,enum,int,real,int,real,int,int,real,int,real,real,real,real,int,real,int,int,real,int,real,int,real,int,real,int,real,int,int,int,int,real,int,real,int,int,real,int,real,int,real,int,int,real,int,int,int,int,real,int,real,int,int,int,real,int,int,int,int,int,int,int,real,int,int,real,int,int,real,int,int,int,real,int,int,real,int,int,int,int,int,real,int,int,int,int,int,int,int,int,real,int,int,int,real,int,int,int,int,real,real,real,real,real,real,real,real,real,real
mins,0.0,0.0,0.0,1918.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.2,-1.3,-0.5,-0.4,-0.3,-0.6,-1.1,-0.1,-0.6,-0.1
maxs,178147.0,11043.0,55.0,1081028538.0,928120.0,3.0,530002.0,3129900.0,76664.0,30857204.7,76664.0,76664.0,30857204.7,76664.0,1426727.4,30857204.7,103847.31,708950.0,13667.0,18598525.9,7170.0,905.0,123330.54,903.0,97225.0,1742.0,4222146.3,1046.0,1740000.0,530002.0,118250.0,105.0,100.0,766.0,764.0,287980.0,752.0,5745385.3,747.0,2661.0,2338699.6,1365.0,3129900.0,544.0,299592.9,544.0,6171.0,3213935.4,2153.0,32.0,0.0,0.0,37800.0,166.0,20750.78,2.0,274000.0,119.0,1041759.0,119.0,40000.0,2.0,0.0,0.0,165000.0,27.0,25000.0,27.0,220.0,188895.49,191.0,43.0,15945.66,33.0,500.0,45.0,25973.23,45.0,224.0,360000.0,224.0,0.0,0.0,0.0,7.0,1254.91,7.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,382.05,2.0,1.0,194.0,93507.0,159.0,0.0,0.0,0.0,1.5,2.1,2.0,1.5,4.1,0.9,1.4,0.8,1.1,1.0
mean,NaN,NaN,NaN,1554203.5,386060.0,NaN,26.7,539.6,13.8,10330.6,12.5,13.3,7610.0,10.9,1092.4,6512.0,2.5,209.4,1.2,1262.5,0.7,0.3,21.3,0.1,45.4,0.1,347.5,0.0,35.9,10.1,48.9,0.0,0.0,0.2,0.1,154.2,0.5,781.1,0.3,0.3,63.1,0.1,76.6,0.3,44.2,0.3,0.3,135.7,0.3,0.0,0.0,0.0,3.1,0.0,0.7,0.0,6.3,0.0,5.0,0.0,0.2,0.0,0.0,0.0,1.4,0.0,0.5,0.0,0.0,2.2,0.0,0.0,0.2,0.0,0.0,0.0,0.3,0.0,0.0,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.1,0.0,0.0,0

In [ ]:
# Split WHD data into test/train with 20/80 ratio
train_mod = whd_arch[split <= 0.8]
test_mod = whd_arch[split > 0.8]

# Build a GBM model to predict repeat violators and score
s = time.time()
dl_mod = H2ODeepLearningEstimator(epochs = 0.1, hidden = [50,50,50], distribution = "multinomial")

dl_mod.train(x               =idx_x,
             y               ="flsa_repeat_violator",
             training_frame  =train,
             validation_frame=test)

mod_elapsed = time.time() - s


deeplearning Model Build Progress: [##################################################] 100%


In [ ]:
# Model performance comparison
train_ll_orig = dl_orig.model_performance(train).logloss()
test_ll_orig  = dl_orig.model_performance(test ).logloss()
train_ll_mod  = dl_mod .model_performance(train).logloss()
test_ll_mod   = dl_mod .model_performance(test ).logloss()

# Print results in pretty HTML table
header = ["Metric"   , "Original"    , "Reduced"    ]
table = [
         ["Runtime"  , orig_elapsed  , mod_elapsed  ],
         ["Train LogLoss", train_ll_orig, train_ll_mod],
         ["Test LogLoss" , test_ll_orig , test_ll_mod ],
        ]
h2o.H2ODisplay(table,header)